In [1]:
import mlearn_cuda as mlearn
from mlearn_cuda import Module
from mlearn_cuda import layers
from mlearn_cuda.optimizer import RMSProp,SGD
from mlearn_cuda.utils import DataLoader,pre_F
from mlearn_cuda import functional as F
from torchvision import datasets
import cupy as cp
import sys

In [18]:
train = datasets.FashionMNIST("datasets/",train=True,download=True)
test = datasets.FashionMNIST("datasets/",train=False,download=True)

In [19]:
pre = [pre_F.normalize_MinMax]
trainset = DataLoader((train.data,train.targets),batch_size=32, shuffle=True, preprocessing = pre)
testset = DataLoader((test.data,test.targets),batch_size=32,shuffle=True, preprocessing=pre)

In [24]:
class Net(mlearn.Module):
    def __init__(self):
        self.dense1 = layers.Dense(784,390)
        self.dense2 = layers.Dense(390,128)
        self.dense3 = layers.Dense(128,10)
        
    def forward(self, inputs):
        o = self.dense1(inputs)
        o = F.relu(o)
        o = self.dense2(o)
        o = F.tanh(o)
        o = self.dense3(o)
        return (F.tanh(o))
net = Net()

In [25]:
hist = []
import numpy as np
optimizer = RMSProp(net,0.001)
EPOCHS = 5
net.zero_grad()
for epoch in range(EPOCHS):
    running_loss = 0.0
    bar = " "*20
    for i, batch in enumerate(trainset, 0):
        features, labels = batch
        net.zero_grad()
        predict = net(features.reshape(-1,784))
        loss = F.cross_entropy(predict,labels)
        loss.backward()
        shapes = optimizer.step()
        running_loss += loss.data
        print(f"\r{epoch+1}/{EPOCHS} Batch %-4d/1874  [{bar}] -> Loss %.5f"%(i,loss.data), end="")
        sys.stdout.flush()
    print()
print('trainning completed!')

1/5 Batch 1874/1562  [                    ] -> Loss 1.08480
2/5 Batch 1874/1562  [                    ] -> Loss 1.14640
3/5 Batch 1874/1562  [                    ] -> Loss 1.10980
4/5 Batch 1874/1562  [                    ] -> Loss 1.10663
5/5 Batch 1874/1562  [                    ] -> Loss 1.21548
trainning completed!


In [28]:
correct = 0
total = 0
features = 0
labels = 0
for batch in testset:
    features, labels = batch
    o = net(features.reshape(-1,784))
    predict = []
    for x in o.data:
        predict.append(cp.argmax(x))
    for b in predict == cp.asnumpy(labels.data):
        if b:
            correct += 1
    total += 32
print("Accuracy %.5f"%(correct / total))

Accuracy 0.79343
